### Dataset: LastFM
### Algorithm: RecentPop
### Padding unknown user/item: N.A.
### Flow: Library Flow

In [1]:
k = 100
from streamsight.datasets.lastfm import LastFMDataset
from streamsight.settings import SlidingWindowSetting
dataset = LastFMDataset()

data = dataset.load()

setting_window = SlidingWindowSetting(
    background_t=1167550546,
    window_size=60 * 60 * 24 * 365, # 365 days
    top_K=k
)

setting_window.split(data)

INFO - streamsight package loaded.
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

DEBUG - LastFMDataset being initialized with 'data' as the base path.
DEBUG - LastFMDataset is initialized.
INFO - LastFMDataset is loading dataset...
DEBUG - Data zipfile is in memory and in dir specified.
DEBUG - LastFMDataset applying filters set.
DEBUG - 	interactions before preprocess: 186479
DEBUG - 	items before preprocess: 12523
DEBUG - 	users before preprocess: 1892
DEBUG - 	interactions after preprocess: 186479
DEBUG - 	items after preprocess: 12523
DEBUG - 	users after preprocess: 1892
INFO - LastFMDataset dataset loaded - Took 0.0917s
DEBUG - Splitting data...
DEBUG - Performing lt(t, 2147483647)
DEBUG - Performing lt(t, 1167550546)
DEBUG - Performing ge

  0%|          | 0/4 [00:00<?, ?it/s]

DEBUG - NPastInteractionTimestampSplitter(t=1167550546,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=1167550546
DEBUG - Performing lt(t, 1199086546)
DEBUG - Performing ge(t, 1167550546)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=1167550546,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 1167550546 resulted in empty unlabelled testing samples.
DEBUG - NPastInteractionTimestampSplitter(t=1167550546,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=1199086546
DEBUG - Performing lt(t, 1230622546)
DEBUG - Performing ge(t, 1199086546)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=1199086546,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 1199086546 re

5it [00:00, 51.81it/s]               

INFO - Finished split with window size 31536000 seconds. Number of splits: 5 in total.
INFO - SlidingWindowSetting data split - Took 0.122s
DEBUG - Checking split attribute and sizes.
DEBUG - Checking split attributes.
DEBUG - Split attributes are set.
DEBUG - Checking size of split sets.
DEBUG - Size of split sets are checked.
INFO - SlidingWindowSetting data split complete.


In [2]:
from streamsight.evaluators import EvaluatorStreamerBuilder

builder = EvaluatorStreamerBuilder()
builder.add_setting(setting_window)
builder.set_metric_K(k)
builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
builder.add_metric("NDCGK")
evaluator = builder.build()

In [3]:
from streamsight.algorithms import RecentPopularity
external_model = RecentPopularity(K=k)
external_model_id = evaluator.register_algorithm(algorithm_name="LastFM_RecentPop")
print(external_model_id)

INFO - Registering algorithm name LastFM_RecentPop with ID: bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d registered
bdd640fb-0667-4ad1-9c80-317fa3b1799d


In [4]:
evaluator.start_stream()

DEBUG - Resetting data generators.
DEBUG - Data generators are reset.
DEBUG - Preparing evaluator for streaming
DEBUG - (user x item) shape defined is (187, 3034)
DEBUG - Shape of dataframe stored in matrix was (12302, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (12302, 5) after masking
DEBUG - Final (user x item) shape defined is (187, 3034)
DEBUG - Caching evaluation data for step 1
DEBUG - (user x item) shape defined is (187, 3034)
DEBUG - Shape of dataframe stored in matrix was (11440, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (11440, 5) after masking
DEBUG - Final (user x item) shape defined is (187, 3034)
DEBUG - (user x item) shape defined is (187, 3034)
DEBUG - Shape of dataframe stored in matrix was (11440, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (7817, 5) after masking
DEBUG - Final (user x item) shape defined is (457, 3034)
DEBUG - Data cached for step 1 complete


# Iteration 1

In [5]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
Sorted scores recentpop:  [0. 0. 0. ... 0. 0. 0.]
DEBUG - Fitting RecentPopularity complete - Took 0.00209s


       interactionid  uid   iid          ts  tagID
0                  0    0     0  -428720400     39
1                  1    1     1  -420771600    103
2                  2    1     1  -420771600  10021
3                  3    2     2  -405133200   1474
4                  4    3     3   294357600    311
...              ...  ...   ...         ...    ...
12297          12297    3  1970  1164927600    102
12298          12298    3  1791  1164927600     81
12299          12299   99  3033  1164927600  11934
12300          12300    3  1791  1164927600     33
12301          12301    3   183  1164927600    387

[12302 rows x 5 columns]

In [6]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [24, 60, 76, 9, 187, 127, 188, 189, 3, 134, 131, 35, 190, 117, 159, 124, 191, 192, 161, 80, 193, 132, 194, 195, 154, 196, 197, 91, 198, 199, 171, 55, 49, 200, 201, 32, 168, 105, 48, 43, 85, 202, 203, 179, 68, 33, 54, 204, 205, 206, 59, 108, 207, 99, 12, 15, 162, 92, 120, 208, 95, 51, 89, 90, 83, 167, 150, 172, 47, 148, 209, 67, 175, 210, 111, 211, 126, 82, 146, 65, 212, 213, 160, 115, 214, 173, 215, 106, 100, 42, 216, 20, 217, 218, 219, 58, 69, 183, 178, 220, 221, 222, 223, 25, 224, 97, 225, 121, 226, 227, 139, 155, 152, 98, 36, 66, 228, 229, 230, 231, 164, 232, 233, 174, 234, 182, 235, 236, 237, 238, 239, 240, 241, 113, 141, 242, 243, 244, 140, 245, 246, 247, 248, 249, 74, 180, 250, 251, 77, 252, 253, 254, 29, 255, 256, 128, 72, 257, 258, 259, 260, 142, 261, 262, 263, 264, 37, 265, 266, 96, 267, 268, 269, 170, 270, 271, 272, 273, 274, 275, 276, 71, 277, 278,

       interactionid  uid  iid          ts  tagID
12302          12302   24   -1  1167606000     76
12303          12303   60   -1  1167606000   1316
12304          12304   60   -1  1167606000   2664
12305          12305   76   -1  1167606000   1844
12306          12306    9   -1  1167606000   7263
...              ...  ...  ...         ...    ...
32693          32693  457   -1  1196463600   4397
32694          32694  457   -1  1196463600   1911
32695          32695  200   -1  1196463600    855
32700          32700  315   -1  1196463600     81
32709          32709  457   -1  1196463600     79

[11440 rows x 5 columns]

In [7]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 39700 stored elements and shape (458, 3034)>
  Coords	Values
  (3, 5)	0.5
  (3, 9)	0.5
  (3, 12)	0.5625
  (3, 13)	0.5
  (3, 23)	0.4375
  (3, 26)	0.4375
  (3, 27)	0.4375
  (3, 28)	0.5
  (3, 29)	0.5
  (3, 30)	0.875
  (3, 31)	0.75
  (3, 32)	0.625
  (3, 34)	0.5
  (3, 39)	0.875
  (3, 40)	0.4375
  (3, 44)	0.8125
  (3, 49)	0.5
  (3, 51)	0.875
  (3, 56)	0.625
  (3, 58)	0.5625
  (3, 61)	0.8125
  (3, 65)	0.5
  (3, 66)	0.625
  (3, 67)	0.6875
  (3, 74)	0.5625
  :	:
  (457, 668)	0.5625
  (457, 717)	0.8125
  (457, 730)	0.4375
  (457, 734)	0.5
  (457, 735)	0.75
  (457, 736)	0.4375
  (457, 756)	0.6875
  (457, 766)	0.625
  (457, 782)	0.5
  (457, 795)	0.4375
  (457, 817)	0.625
  (457, 868)	0.4375
  (457, 900)	0.5
  (457, 935)	0.5
  (457, 999)	0.6875
  (457, 1001)	0.625
  (457, 1095)	0.625
  (457, 1106)	0.75
  (457, 1180)	0.4375
  (457, 1181)	0.5
  (457, 1248)	0.75
  (457, 1504)	0.5
  (457, 1853)	0.6875
  (457, 2032)	0.5
  (457, 2130)	0.4375
[

# Iteration 2

In [8]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (458, 5278)
DEBUG - Shape of dataframe stored in matrix was (20415, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (20415, 5) after masking
DEBUG - Final (user x item) shape defined is (458, 5278)
DEBUG - Caching evaluation data for step 2
DEBUG - (user x item) shape defined is (458, 5278)
DEBUG - Shape of dataframe stored in matrix was (21359, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (21359, 5) after masking
DEBUG - Final (user x item) shape defined is (458, 5278)
DEBUG - (user x item) shape defined is (458, 5278)
DEBUG - Shape of dataframe stored in matrix was (21359, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (15787, 5) after masking
DEBUG - Final (user x item) shape defined is (918, 5278)
DEBUG - Data cached for step 2 complete
Sorted scores recentpop:  [0. 0. 0. ... 0. 0. 0.]
DEBUG - Fitting Recen

       interactionid  uid   iid          ts  tagID
12302          12302   24  3034  1167606000     76
12303          12303   60  3035  1167606000   1316
12304          12304   60  3035  1167606000   2664
12305          12305   76  3036  1167606000   1844
12306          12306    9  1224  1167606000   7263
...              ...  ...   ...         ...    ...
32712          32712  340   512  1196463600    105
32713          32713  419  2309  1196463600     72
32714          32714  340   512  1196463600    570
32715          32715  131  5275  1196463600     47
32716          32716  131  5264  1196463600    187

[20415 rows x 5 columns]

In [9]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [181, 419, 398, 458, 416, 459, 460, 175, 405, 25, 24, 385, 176, 196, 461, 462, 432, 114, 67, 242, 417, 463, 117, 464, 313, 465, 266, 466, 467, 468, 85, 426, 132, 248, 370, 140, 469, 470, 441, 471, 413, 415, 342, 367, 442, 1, 93, 472, 473, 161, 143, 474, 42, 101, 272, 236, 475, 428, 102, 124, 127, 65, 182, 329, 43, 9, 71, 379, 207, 3, 32, 396, 146, 205, 280, 304, 476, 352, 340, 383, 195, 439, 20, 414, 99, 436, 440, 360, 44, 318, 433, 250, 259, 341, 350, 477, 362, 454, 478, 52, 131, 479, 480, 481, 447, 482, 429, 150, 322, 456, 315, 296, 365, 388, 483, 170, 83, 348, 484, 485, 407, 307, 120, 364, 219, 51, 68, 279, 91, 177, 105, 320, 191, 486, 89, 243, 270, 487, 244, 325, 390, 63, 55, 239, 264, 80, 488, 489, 490, 299, 399, 183, 36, 406, 234, 443, 410, 491, 409, 391, 437, 356, 492, 493, 494, 324, 430, 495, 496, 126, 497, 192, 220, 498, 499, 233, 346, 351, 282, 400,

       interactionid  uid  iid          ts  tagID
32717          32717  181   -1  1199142000   3364
32718          32718  419   -1  1199142000     78
32719          32719  419   -1  1199142000    169
32720          32720  398   -1  1199142000     39
32721          32721  458   -1  1199142000     47
...              ...  ...  ...         ...    ...
72968          72968  780   -1  1228086000     25
72971          72971  862   -1  1228086000    515
72973          72973  823   -1  1228086000   7938
72974          72974   91   -1  1228086000   1835
72988          72988  460   -1  1228086000     83

[21359 rows x 5 columns]

In [10]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 74600 stored elements and shape (918, 5278)>
  Coords	Values
  (1, 9)	0.30303030303030304
  (1, 12)	0.5454545454545454
  (1, 14)	0.3939393939393939
  (1, 26)	0.3333333333333333
  (1, 27)	0.36363636363636365
  (1, 29)	0.36363636363636365
  (1, 30)	0.36363636363636365
  (1, 31)	0.6060606060606061
  (1, 32)	0.42424242424242425
  (1, 39)	0.3333333333333333
  (1, 44)	0.5454545454545454
  (1, 51)	0.5151515151515151
  (1, 54)	0.3333333333333333
  (1, 56)	0.36363636363636365
  (1, 58)	0.6060606060606061
  (1, 61)	1.0
  (1, 62)	0.48484848484848486
  (1, 65)	0.5757575757575758
  (1, 66)	0.45454545454545453
  (1, 67)	0.42424242424242425
  (1, 74)	0.3333333333333333
  (1, 89)	0.36363636363636365
  (1, 105)	0.3333333333333333
  (1, 114)	0.5757575757575758
  (1, 123)	0.8181818181818182
  :	:
  (917, 988)	0.45454545454545453
  (917, 999)	0.42424242424242425
  (917, 1001)	0.9696969696969697
  (917, 1095)	0.3939393939393939
  (917, 1106)	0.4

# Iteration 3

In [11]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (918, 8205)
DEBUG - Shape of dataframe stored in matrix was (40273, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (40273, 5) after masking
DEBUG - Final (user x item) shape defined is (918, 8205)
DEBUG - Caching evaluation data for step 3
DEBUG - (user x item) shape defined is (918, 8205)
DEBUG - Shape of dataframe stored in matrix was (23605, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (23605, 5) after masking
DEBUG - Final (user x item) shape defined is (918, 8205)
DEBUG - (user x item) shape defined is (918, 8205)
DEBUG - Shape of dataframe stored in matrix was (23605, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (19148, 5) after masking
DEBUG - Final (user x item) shape defined is (1369, 8205)
DEBUG - Data cached for step 3 complete
Sorted scores recentpop:  [0. 0. 0. ... 0. 0. 0.]
DEBUG - Fitting Rece

       interactionid  uid   iid          ts  tagID
32717          32717  181  1332  1199142000   3364
32718          32718  419  5278  1199142000     78
32719          32719  419  5278  1199142000    169
32720          32720  398  1080  1199142000     39
32721          32721  458  5279  1199142000     47
...              ...  ...   ...         ...    ...
72985          72985  639   744  1228086000   8504
72986          72986  502  8204  1228086000    109
72987          72987  502  8200  1228086000    724
72988          72988  460  2139  1228086000     83
72989          72989  502  8200  1228086000    109

[40273 rows x 5 columns]

In [12]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [918, 412, 460, 196, 734, 661, 703, 739, 820, 68, 547, 874, 462, 343, 132, 404, 575, 813, 919, 413, 528, 283, 920, 802, 598, 435, 921, 407, 922, 923, 376, 694, 391, 924, 493, 45, 925, 776, 57, 124, 373, 639, 294, 259, 478, 127, 763, 117, 822, 862, 352, 441, 926, 714, 881, 917, 203, 927, 825, 821, 464, 826, 159, 85, 650, 891, 307, 928, 815, 842, 288, 728, 711, 4, 764, 929, 910, 502, 264, 81, 255, 741, 789, 91, 361, 911, 367, 901, 330, 736, 735, 930, 667, 931, 807, 622, 590, 774, 415, 500, 421, 167, 812, 148, 266, 932, 852, 933, 671, 190, 934, 839, 935, 370, 936, 707, 705, 140, 219, 533, 47, 233, 937, 816, 536, 733, 938, 541, 254, 115, 234, 399, 618, 186, 939, 29, 869, 754, 870, 773, 439, 120, 889, 793, 792, 636, 940, 499, 941, 340, 426, 785, 883, 942, 643, 484, 280, 414, 457, 642, 943, 318, 551, 637, 900, 887, 801, 131, 635, 83, 250, 99, 944, 282, 398, 428, 27

        interactionid   uid  iid          ts  tagID
72990           72990   918   -1  1230764400    176
72991           72991   412   -1  1230764400    275
72992           72992   918   -1  1230764400    227
72993           72993   460   -1  1230764400     15
72994           72994   196   -1  1230764400    187
...               ...   ...  ...         ...    ...
116338         116338   456   -1  1259622000    693
116341         116341   207   -1  1259622000    387
116347         116347   207   -1  1259622000    191
116348         116348  1339   -1  1259622000   4228
116351         116351   456   -1  1259622000    432

[23605 rows x 5 columns]

In [13]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 90900 stored elements and shape (1369, 8205)>
  Coords	Values
  (1, 12)	0.6037735849056604
  (1, 13)	0.2830188679245283
  (1, 29)	0.3584905660377358
  (1, 31)	0.4339622641509434
  (1, 32)	0.32075471698113206
  (1, 39)	0.4339622641509434
  (1, 44)	0.37735849056603776
  (1, 51)	0.49056603773584906
  (1, 56)	0.660377358490566
  (1, 58)	0.6037735849056604
  (1, 61)	0.7358490566037735
  (1, 62)	0.39622641509433965
  (1, 65)	0.4528301886792453
  (1, 67)	0.4716981132075472
  (1, 74)	0.3584905660377358
  (1, 78)	0.3018867924528302
  (1, 84)	0.3584905660377358
  (1, 86)	0.32075471698113206
  (1, 89)	0.3018867924528302
  (1, 105)	0.2830188679245283
  (1, 113)	0.32075471698113206
  (1, 114)	0.8490566037735849
  (1, 123)	0.8867924528301887
  (1, 130)	0.4716981132075472
  (1, 140)	0.3018867924528302
  :	:
  (1368, 1181)	0.32075471698113206
  (1368, 1248)	0.4528301886792453
  (1368, 1300)	0.32075471698113206
  (1368, 1355)	0.3207547169811

# Iteration 4

In [14]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (1369, 10380)
DEBUG - Shape of dataframe stored in matrix was (43366, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (43366, 5) after masking
DEBUG - Final (user x item) shape defined is (1369, 10380)
DEBUG - Caching evaluation data for step 4
DEBUG - (user x item) shape defined is (1369, 10380)
DEBUG - Shape of dataframe stored in matrix was (27243, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (27243, 5) after masking
DEBUG - Final (user x item) shape defined is (1369, 10380)
DEBUG - (user x item) shape defined is (1369, 10380)
DEBUG - Shape of dataframe stored in matrix was (27243, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (23382, 5) after masking
DEBUG - Final (user x item) shape defined is (1821, 10380)
DEBUG - Data cached for step 4 complete
Sorted scores recentpop:  [0. 0. 0. ... 0. 0. 0.]
DEBUG - F

        interactionid   uid   iid          ts  tagID
72990           72990   918  2027  1230764400    176
72991           72991   412  1223  1230764400    275
72992           72992   918  5332  1230764400    227
72993           72993   460  2144  1230764400     15
72994           72994   196  1446  1230764400    187
...               ...   ...   ...         ...    ...
116351         116351   456  1306  1259622000    432
116352         116352   739   376  1259622000     16
116353         116353  1301  1095  1259622000    847
116354         116354  1115   736  1259622000    134
116355         116355   428  3980  1259622000   8304

[43366 rows x 5 columns]

In [15]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [220, 1369, 1194, 1219, 183, 1257, 1025, 964, 998, 1222, 1039, 1115, 918, 132, 965, 1259, 396, 584, 1282, 1370, 1258, 958, 856, 1365, 703, 1049, 1297, 924, 1112, 1212, 1371, 1058, 1229, 1347, 1294, 1226, 1228, 1167, 1168, 384, 193, 1298, 1299, 1372, 1308, 503, 1195, 372, 793, 1358, 207, 1307, 528, 391, 766, 774, 1310, 1199, 853, 51, 562, 1089, 873, 43, 1044, 1035, 772, 1340, 927, 1355, 943, 678, 1087, 177, 1084, 1274, 1136, 1201, 680, 544, 1248, 1373, 1311, 1245, 1169, 745, 1091, 282, 210, 1114, 1374, 1357, 231, 900, 1368, 1033, 736, 17, 1100, 1361, 1375, 1230, 734, 462, 1099, 1107, 24, 1376, 800, 1278, 1071, 454, 795, 739, 1020, 1235, 920, 1322, 1237, 1377, 460, 547, 1070, 1378, 1379, 845, 1, 1380, 940, 1381, 1279, 821, 971, 233, 127, 1241, 942, 741, 977, 765, 447, 712, 1288, 1051, 148, 1005, 124, 1382, 1321, 914, 1316, 83, 707, 747, 1332, 705, 706, 1232, 10

        interactionid   uid  iid          ts  tagID
116356         116356   220   -1  1262300400    386
116357         116357   220   -1  1262300400    387
116358         116358   220   -1  1262300400    147
116359         116359   220   -1  1262300400    386
116360         116360   220   -1  1262300400     72
...               ...   ...  ...         ...    ...
171349         171349  1815   -1  1291158000     73
171350         171350  1815   -1  1291158000     81
171351         171351  1801   -1  1291158000    275
171352         171352  1815   -1  1291158000    134
171353         171353  1698   -1  1291158000     16

[27243 rows x 5 columns]

In [16]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 102900 stored elements and shape (1821, 10380)>
  Coords	Values
  (0, 12)	0.42028985507246375
  (0, 29)	0.30434782608695654
  (0, 30)	0.4927536231884058
  (0, 31)	0.4057971014492754
  (0, 32)	0.2318840579710145
  (0, 39)	0.2608695652173913
  (0, 51)	0.4057971014492754
  (0, 54)	0.2898550724637681
  (0, 55)	0.2608695652173913
  (0, 56)	0.5362318840579711
  (0, 58)	0.5072463768115942
  (0, 61)	0.7681159420289855
  (0, 62)	0.2463768115942029
  (0, 65)	0.5507246376811594
  (0, 66)	0.34782608695652173
  (0, 67)	0.42028985507246375
  (0, 105)	0.2608695652173913
  (0, 114)	0.6376811594202898
  (0, 123)	0.6376811594202898
  (0, 130)	0.2318840579710145
  (0, 147)	0.37681159420289856
  (0, 150)	0.391304347826087
  (0, 151)	0.37681159420289856
  (0, 153)	0.43478260869565216
  (0, 156)	0.6956521739130435
  :	:
  (1820, 1266)	0.2318840579710145
  (1820, 1300)	0.5507246376811594
  (1820, 1301)	0.2753623188405797
  (1820, 1355)	0.260869565

# Iteration 5

In [17]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (1821, 12146)
DEBUG - Shape of dataframe stored in matrix was (54998, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (54998, 5) after masking
DEBUG - Final (user x item) shape defined is (1821, 12146)
DEBUG - Caching evaluation data for step 5
DEBUG - (user x item) shape defined is (1821, 12146)
DEBUG - Shape of dataframe stored in matrix was (10224, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (10224, 5) after masking
DEBUG - Final (user x item) shape defined is (1821, 12146)
DEBUG - (user x item) shape defined is (1821, 12146)
DEBUG - Shape of dataframe stored in matrix was (10224, 5) before masking
DEBUG - Shape of dataframe stored in matrix is now (9342, 5) after masking
DEBUG - Final (user x item) shape defined is (1890, 12146)
DEBUG - Data cached for step 5 complete
Sorted scores recentpop:  [0. 0. 0. ... 0. 0. 0.]
DEBUG - Fi

        interactionid   uid    iid          ts  tagID
116356         116356   220   1473  1262300400    386
116357         116357   220    528  1262300400    387
116358         116358   220   1554  1262300400    147
116359         116359   220   2352  1262300400    386
116360         116360   220  10380  1262300400     72
...               ...   ...    ...         ...    ...
171349         171349  1815  10021  1291158000     73
171350         171350  1815  10021  1291158000     81
171351         171351  1801     30  1291158000    275
171352         171352  1815  10021  1291158000    134
171353         171353  1698    202  1291158000     16

[54998 rows x 5 columns]

In [18]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Performing items_in comparison
Users:  [853, 1801, 1186, 1404, 1821, 1195, 1569, 1308, 1392, 1639, 1693, 1737, 251, 528, 1549, 1487, 702, 772, 1822, 1276, 1050, 1356, 1790, 1041, 101, 1823, 1699, 693, 1027, 1804, 1824, 865, 1146, 1825, 1784, 1696, 768, 943, 1274, 1201, 282, 602, 1808, 1498, 1670, 1780, 1826, 300, 1827, 699, 586, 1464, 1781, 1679, 1828, 1692, 1354, 1732, 1033, 1726, 1102, 1514, 1402, 869, 1578, 1829, 1233, 1702, 1545, 1466, 672, 1260, 1776, 1629, 1715, 1261, 1608, 1284, 633, 1635, 1472, 434, 1830, 1017, 1316, 821, 1816, 1390, 1800, 724, 828, 1775, 1387, 1774, 1785, 1229, 1730, 905, 1770, 1237, 1786, 1112, 776, 1733, 460, 1293, 196, 1443, 547, 1550, 1254, 1712, 1711, 656, 1253, 918, 1259, 503, 352, 1814, 842, 1717, 1600, 1104, 1831, 1813, 1480, 1716, 1475, 1278, 1222, 1641, 965, 1445, 0, 1298, 464, 1297, 1806, 1832, 1352, 1833, 1615, 1265, 1654, 297, 1719, 1001, 141, 721, 1444, 1080

        interactionid   uid  iid          ts  tagID
171354         171354   853   -1  1293836400     24
171355         171355   853   -1  1293836400    130
171356         171356  1801   -1  1293836400    275
171357         171357  1186   -1  1293836400    424
171358         171358  1404   -1  1293836400     39
...               ...   ...  ...         ...    ...
186445         186445  1237   -1  1304940155    389
186446         186446  1237   -1  1304940165     80
186447         186447  1237   -1  1304940174    481
186448         186448  1237   -1  1304940176    121
186449         186449  1237   -1  1304940176    121

[10224 rows x 5 columns]

In [19]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 55700 stored elements and shape (1892, 12146)>
  Coords	Values
  (0, 12)	0.2736842105263158
  (0, 30)	0.30526315789473685
  (0, 31)	0.22105263157894736
  (0, 32)	0.3263157894736842
  (0, 39)	0.24210526315789474
  (0, 51)	0.21052631578947367
  (0, 56)	0.8631578947368421
  (0, 58)	0.35789473684210527
  (0, 61)	0.5894736842105263
  (0, 62)	0.21052631578947367
  (0, 65)	0.43157894736842106
  (0, 66)	0.3473684210526316
  (0, 67)	0.4
  (0, 105)	0.21052631578947367
  (0, 114)	0.5684210526315789
  (0, 123)	0.4631578947368421
  (0, 130)	0.21052631578947367
  (0, 147)	0.42105263157894735
  (0, 150)	0.2631578947368421
  (0, 151)	0.21052631578947367
  (0, 153)	0.3157894736842105
  (0, 156)	0.6105263157894737
  (0, 158)	0.3263157894736842
  (0, 161)	0.23157894736842105
  (0, 199)	0.28421052631578947
  :	:
  (1891, 2497)	0.2
  (1891, 2873)	0.21052631578947367
  (1891, 3578)	0.2
  (1891, 3593)	0.23157894736842105
  (1891, 3662)	0.263157894

# evaluate metrics

In [20]:
evaluator.get_all_algorithm_status()

{'LastFM_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.COMPLETED: 'COMPLETED'>}

In [21]:
evaluator.metric_results("macro")

macro_score  \
Algorithm                                          Metric                        
LastFM_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3... NDCGK_100          0.106884   
                                                   PrecisionK_100     0.019087   
                                                   RecallK_100        0.221893   

                                                                   num_window  
Algorithm                                          Metric                      
LastFM_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3... NDCGK_100                5  
                                                   PrecisionK_100           5  
                                                   RecallK_100              5

In [22]:
evaluator.metric_results("micro")


micro_score  \
Algorithm                                          Metric                       
LastFM_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3... NDCGK_100         0.107668   
                                                   PrecisionK_100    0.018985   
                                                   RecallK_100       0.219911   

                                                                   num_user  
Algorithm                                          Metric                    
LastFM_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3... NDCGK_100           3282  
                                                   PrecisionK_100      3282  
                                                   RecallK_100         3282

In [23]:
evaluator.metric_results("user")

user_id  \
Algorithm                                          Timestamp    Metric                   
LastFM_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3... t=1167550546 PrecisionK_100       3   
                                                                PrecisionK_100       4   
                                                                PrecisionK_100       8   
                                                                PrecisionK_100       9   
                                                                PrecisionK_100      12   
...                                                                                ...   
                                                   t=1293694546 NDCGK_100         1885   
                                                                NDCGK_100         1886   
                                                                NDCGK_100         1887   
                                                                NDCGK_100         1888   
                                                                NDCGK_100         1889   

                                                                                   score  
Algorithm                                          Timestamp    Metric                    
LastFM_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3... t=1167550546 PrecisionK_100      0.01  
                                                                PrecisionK_100       0.0  
                                                                PrecisionK_100       0.0  
                                                                PrecisionK_100      0.04  
                                                                PrecisionK_100       0.0  
...                                                                                  ...  
                                                   t=1293694546 NDCGK_100       0.176134  
                                                                NDCGK_100            0.0  
                                                                NDCGK_100       0.348467  
                                                                NDCGK_100       0.144246  
                                                                NDCGK_100            0.0  

[9846 rows x 2 columns]

In [24]:
evaluator.metric_results("window")

window_score  \
Algorithm                                          Timestamp    Metric                         
LastFM_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3... t=1167550546 PrecisionK_100      0.024129   
                                                                RecallK_100         0.243154   
                                                                NDCGK_100           0.111540   
                                                   t=1199086546 PrecisionK_100      0.018982   
                                                                RecallK_100         0.187875   
                                                                NDCGK_100           0.091728   
                                                   t=1230622546 PrecisionK_100      0.019292   
                                                                RecallK_100         0.207217   
                                                                NDCGK_100           0.102381   
                                                   t=1262158546 PrecisionK_100      0.020256   
                                                                RecallK_100         0.235744   
                                                                NDCGK_100           0.122993   
                                                   t=1293694546 PrecisionK_100      0.012776   
                                                                RecallK_100         0.235476   
                                                                NDCGK_100           0.105777   

                                                                                num_user  
Algorithm                                          Timestamp    Metric                    
LastFM_RecentPop_bdd640fb-0667-4ad1-9c80-317fa3... t=1167550546 PrecisionK_100       356  
                                                                RecallK_100          356  
                                                                NDCGK_100            356  
                                                   t=1199086546 PrecisionK_100       658  
                                                                RecallK_100          658  
                                                                NDCGK_100            658  
                                                   t=1230622546 PrecisionK_100       805  
                                                                RecallK_100          805  
                                                                NDCGK_100            805  
                                                   t=1262158546 PrecisionK_100       937  
                                                                RecallK_100          937  
                                                                NDCGK_100            937  
                                                   t=1293694546 PrecisionK_100       526  
                                                                RecallK_100          526  
                                                                NDCGK_100            526